In [13]:
import pandas as pd

# import scipy
# from scipy.stats import t
# import seaborn as sns
# from datetime import datetime, timedelta
# import plotly.express as px
# from pathlib import Path
# import statsmodels.api as sm
# from sklearn.preprocessing import StandardScaler
# from statsmodels.stats.multitest import multipletests
# from scipy.stats import f_oneway as anova
# from statsmodels.formula.api import ols 
  
# import math
# from select import select

#kernal must be restarted if these module change 

from SCP_processor import SCP_processor
processor = SCP_processor(data_type="TMT")
from SCP_plotter import SCP_plotter
plotter = SCP_plotter(write_output = True,processor=processor)

In [14]:
#Method put files in, one dictionary for each analysis, if there is only one file or you are looking in all files, then you don't need @'s in your SETTINGS_FILE
WRITE_OUTPUT = True
USE_MaxLFQ = False
filelist = [
             {"input1":"input/tmt-report/abundance_protein_MD.tsv",
             "input2":"input/tmt-report/abundance_peptide_MD.tsv",
             "input3":"report.pg_matrix.tsv",
             "input4":"report.pr_matrix.tsv",
             "input5":"summary.txt",
             "process_app": "FragPipe_TMT"},
          
            ]

            #Order of files
                #For FragPipe
                    #input1= Protein with MBR
                    #input2= Peptide with MBR
                #For DIANN
                    #input1= pg_matrix
                    #input2= pr_matrix 
                    #input3= pg_matrix MS2
                    #input4= pr_matrix MS2
                #For PD
                    #input1= Protein 
                    #input2= Peptide Groups
                    #input3= blank
                    #input4= blank
                    #input5= inpit_files


SETTINGS_FILE = "settings_test_TMT.txt" #tab delimited
    #Format
    #Required Columns
        #Group Name	
        #filter_in: a string pattern contained in desired raw filenames in analysis followed by an @ + the file number (filelist index) to look in
        #filter_out: a string pattern contained in undesired raw filenames
    #Extra columns
        #can be used to group by color, layer, or x position in CV and ID graphs

In [15]:
# define group names and assign analysis to different groups(web app version was done through GUI so
# steps are taken to make sure they have same output)
x = processor.read_files(grouped_input_files=filelist)




In [16]:
data_obj = processor.outer_join_data_objects(x)


***


In [17]:
saved_settings = processor.sort_runs(data_obj,SETTINGS_FILE)

In [18]:
#Missing val plot plot(protein and peptides)
plot_options={
            "mean label": "True",
            "error bar": "ci95", #None, stdev, or ci95
            "X Title": "Conditions",
            "cutoffs": [33.3,50,99],
            "Y Title": "Protein Identifications",
            "color": ["#3E6990", "#7aa8e6","#AABD8C", "#E9E3B4", "#C6878F", "#fac8d3","#F39B6D",  "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
            "width": 700,
            "height": 450,
            "font": dict(size=32, family="Arial black"),
            "ID mode": "stacked",#stacked
            "Group By X": "Conditions", #ID_Mode does MS2 vs MBR
        }
plot_options["plot_type"] = "1" # 1 is protein, 2 is peptide
figure ,_ ,_ =plotter.Missing_Values_Plots(data_obj, plot_options, saved_settings)
figure.write_image("greedy_missing_val.png", scale=10, width = 1000, height=1000)
figure.show()

    Accession 0-1-0 0-1-1 0-1-2 0-1-3 0-1-4 0-1-5 0-1-6 0-1-7 0-1-8 0-1-9  \
0       RPSA2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   nan   
1      EIF3CL   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   nan   
2        NACA   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   nan   
3     SUMO1P1   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   
4        AGPS   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   nan   
..        ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
672     CHTOP   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   
673      TLN1   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   
674     HYOU1   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   
675   SUPT16H   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   
676     RBM8A   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   nan   

     missingValues  
0                0  
1                0  
2           

c:\VS_Code\SCP_PP\SCP_plotter.py:620: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\VS_Code\SCP_PP\SCP_plotter.py:620: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\VS_Code\SCP_PP\SCP_plotter.py:620: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\VS_Code\SCP_PP\SCP_plotter.py:620: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\VS_Code\SCP_PP\SCP_plotter.py:620: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\VS_Code\SCP_PP\SCP_plotter.py:620: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future

In [19]:
#ID plot(protein and peptides)
plot_options={
            "mean label": "True",
            "error bar": "ci95", #None, stdev, or ci95
            "X Title": "Conditions",
            "Y Title": "Protein Identifications",
            "color": ["#3E6990", "#7aa8e6","#AABD8C", "#E9E3B4", "#C6878F", "#fac8d3","#F39B6D",  "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
            "width": 700,
            "height": 450,
            "font": dict(size=32, family="Arial black"),
            "ID mode": "grouped_stacked",#grouped, total, MS2, grouped_stacked, stacked
            "Group By X": "Sample Amount", #ID_Mode does MS2 vs MBR
            "Group By Color": "Processing App",#ID_Mode does MS2 vs MBR
            "Group By Stack": "ID_Mode",#ID_Mode does MS2 vs MBR
        }
plot_options["plot_type"] = "1" # 1 is protein, 2 is peptide
figure ,_ ,_ =plotter.ID_plots(data_obj, plot_options, saved_settings)
figure.write_image("asdf.png", scale=10, width = 1000, height=1000)
figure.show()

In [20]:
# CV violin plot

# print(data_obj["protein_other_info"])
plot_options={    
        "median label": "True", #only works for total and MS2, can be found in CV_summary if you set WRITE_OUTPUT = True
        "box": False,
        "X Title": "Conditions (protein)",
        "Y Title": "CV of Abundance (%)",
        "color": ["#3E6990", "#7aa8e6","#C6878F", "#fac8d3","#AABD8C", "#E9E3B4",  "#F39B6D", "#C6878F", "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
        "width": 700,
        "height": 450,
        "font": dict(size=16, family="Arial black"),
        "ylimits": [-10, 400],
        "CV mode": "total",#grouped, total, MS2, grouped_stacked, stacked
        "Group By X": "Conditions", #ID_Mode does MS2 vs MBR
        "Group By Color": "Acquisition Mode",#ID_Mode does MS2 vs MBR
        "Group By Stack": "ID_Mode" #ID_Mode does MS2 vs MBR
    }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =plotter.CV_plots(data_obj, plot_options, saved_settings)
figure.show()


In [21]:
# ID Venns plot
plot_options={
            "compare groups": ["FP22","FP21"],
            "title": "Venn Diagram (Protein)",
            "opacity": 0.75,
            "color": ["#3E6990", "#C6878F", "#AABD8C","#fac8d3","#7aa8e6", "#E9E3B4","#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ]
        }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =plotter.venns_plots(data_obj, plot_options, saved_settings)
figure.show()



    Accession 0-0-0 0-0-1 0-0-2 0-0-3 0-0-4 0-0-5 0-0-6 0-0-7 0-0-8  ...  \
0       RPSA2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2  ...   
1      EIF3CL   nan   nan   nan   nan   nan   nan   nan   nan   nan  ...   
2        NACA   nan   nan   nan   nan   nan   nan   nan   nan   nan  ...   
3     SUMO1P1   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2  ...   
4        AGPS   nan   nan   nan   nan   nan   nan   nan   nan   nan  ...   
..        ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
672     CHTOP   MS2   MS2   nan   MS2   nan   MS2   nan   nan   nan  ...   
673      TLN1   MS2   MS2   MS2   nan   MS2   MS2   MS2   nan   MS2  ...   
674     HYOU1   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2  ...   
675   SUPT16H   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2  ...   
676     RBM8A   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2   MS2  ...   

    0-2-0 0-2-1 0-2-2 0-2-3 0-2-4 0-2-5 0-2-6 0-2-7 0-2-8 0-2-9  
0     MS2   MS2   MS2

In [22]:
# Protein Ranked Abundance plot
plot_options={
            "reference_group": "FP21",
            "compare groups": ["FP22"],
            "title": "Ranked Abundance",
            "color": ["blue", "red", "green", "black", "yellow", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['circle', 'circle', 'circle'],
            "marker_size": 8,
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": [],
            "X Title": "Ranked Abundance",
            "Y Title": "Average Reporter Ion Abundance",
        }
figure, _, _ =plotter.Rank_Abundance_Plots(data_obj, plot_options, saved_settings)
figure.show()

c:\VS_Code\SCP_PP\SCP_plotter.py:1938: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

c:\VS_Code\SCP_PP\SCP_plotter.py:1938: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\VS_Code\SCP_PP\SCP_plotter.py:1976: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [23]:
# Protein Abundance Volcano plot
plot_options={
            "compare groups": ["FP22","FP21"],
            "title": "Volcano Plot",
            "X Title": "Log2 Fold Change",
            "Y Title": "-Log10(adjusted p-value)",
            "up color": "red",
            "down color": "blue",
            "all color": "gray",
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =plotter.volcano_plots(data_obj, plot_options, saved_settings)
figure.show()


(320,)
(445, 4)
(461, 4)
(320, 4)
(320, 4)
Index(['FP21_Intensity', 'group2_stdev', 'group2_num', 'Accession'], dtype='object')
Index(['FP22_Intensity', 'group1_stdev', 'group1_num', 'Accession'], dtype='object')
0       RPSA2
6      PSMD11
9        IPO5
11     PSMD14
14      DDX3X
        ...  
664    RUVBL1
665      NUDC
667    THRAP3
669    LUC7L2
676     RBM8A
Name: Accession, Length: 320, dtype: object
     FP21_Intensity  group2_stdev  group2_num Accession  FP22_Intensity  \
0         25.827311      0.283759           9     RPSA2       25.858251   
6         20.206812      0.236848           9    MYL12B       20.402975   
9         17.198448      0.337079           7     EIF3H       17.529273   
11        20.448107      0.265404           9     PHGDH       20.827383   
14        19.342838      0.613281           9     NARS1       19.893777   
..              ...           ...         ...       ...             ...   
313       20.922283      0.431979           9     PA2G4       21

In [26]:
# Protein Abundance PCA plot
plot_options={
            "compare groups": ["FP22","FP21"],
            "title": "PCA Analysis",
            "color": ["blue", "red", "green", "black", "yellow", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['circle', 'circle', 'circle'],
            "marker_size": 8,
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =plotter.PCA_plots(data_obj, plot_options, saved_settings)
figure.show()


Empty


In [28]:
# Protein Abundance Heat Map
plot_options={
            "compare groups": ["FP22","FP21"],
            "log2_transform": False,
            "significant_only": False,
            "alpha": 0.05, 
            "min_fold_change": 2
        }
figure, _, _ =plotter.heatmap_plots(data_obj, plot_options, saved_settings)
figure.show()

Empty


AttributeError: Can only use .str accessor with string values!

In [ ]:
# ID Venns plot
plot_options={
            "compare groups": ["Quant","Library"],
            "inclusion list": "proteins_symbols.csv",
            "title": "Venn Diagram (Protein)",
            "opacity": 0.75,
            "color": ["#3E6990", "#C6878F", "#AABD8C","#fac8d3","#7aa8e6", "#E9E3B4","#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ]
        }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =plotter.inclusion_venn_plots(data_obj, plot_options, saved_settings,miss_val_thresh=99)
figure.show()

